In [2]:
import pandas as pd
import json

# Load Excel sheets
dashboard = pd.ExcelFile("data.xlsx")
data = pd.read_excel(dashboard, "Statistics")
aes = pd.read_excel(dashboard, 'AEs')
referrals = pd.read_excel(dashboard, 'Referrals')

partnerDistricts =pd.read_excel("partnerDistrict.xlsx")

# Fill missing values and convert types
data.fillna(0, inplace=True)
aes.fillna(0, inplace=True)
referrals.fillna(0, inplace=True)

if type(referrals['ReferrelRecordingYear'])!= int:
    referrals['ReferrelRecordingYear'] = 2025

data['year'] = data['year'].astype(int)
data['recordingMonth'] = data['recordingMonth'].astype(str)
aes['AERecordingYear'] = aes['AERecordingYear'].astype(int)
aes['AERecordingMonth'] = aes['AERecordingMonth'].astype(str)
referrals['ReferrelRecordingYear'] = referrals['ReferrelRecordingYear'].astype(int)
referrals['ReferrelRecordingMonth'] = referrals['ReferrelRecordingMonth'].astype(str)




# Process rows and build JSON
def process_row(row):
    row_data = row.to_dict()
    if int(pd.to_numeric(row.get('totalAES', 0), errors='coerce')) > 0:
        recordedAEs = aes[
            (aes['AERecordingSite'] == row['facilityName']) &
            (aes['AERecordingMonth'] == row['recordingMonth']) &
            (aes['AERecordingYear'] == row['year'])
        ]

        recordedAEs = recordedAEs[recordedAEs['ae_classification']!=0]

        row_data['matchingAES'] = recordedAEs.to_dict(orient='records')
    else:
        row_data['matchingAES'] = []
            
    if int(pd.to_numeric(row.get('referralToOtherServices', 0), errors='coerce')) > 0:
        otherReferrals = referrals[
            (referrals['ReferrelRecordingSite'] == row['facilityName']) &
            (referrals['ReferrelRecordingMonth'] == row['recordingMonth']) &
            (referrals['ReferrelRecordingYear'] == row['year'])
        ]

        otherReferrals = otherReferrals[otherReferrals['services_referred']!=0]
        # print(otherReferrals)
        row_data['otherReferrals'] = otherReferrals.to_dict(orient='records')
    else:
        print("no data")
        row_data['otherReferrals'] = []

    return row_data

partner_lookup = dict(zip(partnerDistricts['DISTRICT'], partnerDistricts['PARTNER']))

data['Partner'] = data['District'].map(partner_lookup)


json_data = data.apply(process_row, axis=1).tolist()

# Save to JSON
output_file = "data.json"
with open(output_file, "w") as file:
    json.dump(json_data, file, indent=4)

# Print the JSON
print(json.dumps(json_data, indent=4))



no data
no data
no data
no data
no data
[
    {
        "AERecordingMonth": 0,
        "AERecordingSite": 0,
        "AERecordingYear": 0,
        "District": "GOKWE SOUTH",
        "ReferrelRecordingMonth": 0,
        "ReferrelRecordingSite": 0,
        "ReferrelRecordingYear": 0,
        "Site_Name": 0,
        "TotalMCsBYMethod": 19,
        "aeComments": 0,
        "ae_classification": 0,
        "ae_type_code": 0,
        "ageGroupsChange": 0,
        "checkMonthValidation": 0,
        "checkYearValidation": 0,
        "checkifFacilityHasBeenSaved": 0,
        "circumcising_cadre": 0,
        "client_age": 0,
        "date_ae_identified": 0,
        "facilityName": "Chitave Clinic",
        "followUpTotal": 14,
        "formType": "submission",
        "fu15-19": 2,
        "fu20-24": 11,
        "fu25-29": 1,
        "fu30-34": 0,
        "fu35-39": 0,
        "fu40-44": 0,
        "fu45-49": 0,
        "fu50": 0,
        "fuAgeUpdate": 0,
        "high_risk_referrals_to_care": 0

In [3]:
data

,AERecordingMonth,AERecordingSite,AERecordingYear,District,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,Site_Name,TotalMCsBYMethod,aeComments,...,total_mcs_referred_to_other_services,total_surgicalDisposable,total_surgicalReusable,totalhivNegative,totalhivNegativeNC,uncircumcisedClientsForHTS,vmmc_number,year,facilityType,Partner
0,0,0,0,GOKWE SOUTH,0,0,0,0,19,0,...,0,OK,0,14,0,0.0,0,2025,0,ZACH
1,April,MBIRE,submission,MBIRE,0,0,0,0,18,Client is steadly healing after receiving trea...,...,0,OK,OK,18,0,0.0,338,2025,0,ZACH
2,0,0,0,LUPANE,0,0,0,0,10,0,...,0,OK,0,10,0,0.0,0,2025,0,ZACH
3,0,0,0,GOKWE SOUTH,0,0,0,0,8,0,...,0,OK,OK,7,0,0.0,0,2025,0,ZACH
4,0,0,0,HURUNGWE,0,0,0,0,27,0,...,0,OK,OK,27,0,0.0,0,2025,0,ZACH
